# Multiheadattention 이해 요함.
# keras vs torch
# permute 왜 했는지 이해해야함.
# Layer norm ; keras vs torch
-> layer norm 자체도 디테일하게 파라미터를 전부 공유하는지등 이해해야함.
- tf.keras.layers.LayerNormalization tensorflow에 이거 제공함.

# 일단 tensorflow에 동일한 built in method들이 존재해서 그대로 결과를 볼 수 있을 것 같음.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import shutil
import sys
import numpy as np
from scipy import sparse
import matplotlib.pyplot as plt
import seaborn as sn
import pandas as pd
import tensorflow as tf
import bottleneck as bn
import keras
import math

In [ ]:
import argparse
import pickle
import time
import numpy as np

In [ ]:
cd /content/drive/MyDrive/Colab Notebooks/recommendation_system/deep_learning/session_based_rec/TAGNN-master/

In [ ]:
train_data = pickle.load(open('../datasets/' + 'diginetica' + '/train.txt', 'rb'))

In [ ]:

def build_graph(train_data):
    graph = nx.DiGraph()
    for seq in train_data:
        for i in range(len(seq) - 1):
            if graph.get_edge_data(seq[i], seq[i + 1]) is None:
                weight = 1
            else:
                weight = graph.get_edge_data(seq[i], seq[i + 1])['weight'] + 1
            graph.add_edge(seq[i], seq[i + 1], weight=weight)
    for node in graph.nodes:
        sum = 0
        for j, i in graph.in_edges(node):
            sum += graph.get_edge_data(j, i)['weight']
        if sum != 0:
            for j, i in graph.in_edges(i):
                graph.add_edge(j, i, weight=graph.get_edge_data(j, i)['weight'] / sum)
    return graph


def data_masks(all_usr_pois, item_tail):
    us_lens = [len(upois) for upois in all_usr_pois]
    len_max = max(us_lens)
    us_pois = [upois + item_tail * (len_max - le) for upois, le in zip(all_usr_pois, us_lens)]
    us_msks = [[1] * le + [0] * (len_max - le) for le in us_lens]
    return us_pois, us_msks, len_max


def split_validation(train_set, valid_portion):
    train_set_x, train_set_y = train_set
    n_samples = len(train_set_x)
    sidx = np.arange(n_samples, dtype='int32')
    np.random.shuffle(sidx)
    n_train = int(np.round(n_samples * (1. - valid_portion)))
    valid_set_x = [train_set_x[s] for s in sidx[n_train:]]
    valid_set_y = [train_set_y[s] for s in sidx[n_train:]]
    train_set_x = [train_set_x[s] for s in sidx[:n_train]]
    train_set_y = [train_set_y[s] for s in sidx[:n_train]]

    return (train_set_x, train_set_y), (valid_set_x, valid_set_y)


class Data():
    def __init__(self, data, shuffle=False, graph=None):
        inputs = data[0]
        inputs, mask, len_max = data_masks(inputs, [0])
        self.inputs = np.asarray(inputs)
        self.mask = np.asarray(mask)
        self.len_max = len_max
        self.targets = np.asarray(data[1])
        self.length = len(inputs)
        self.shuffle = shuffle
        self.graph = graph

    def generate_batch(self, batch_size):
        if self.shuffle:
            shuffled_arg = np.arange(self.length)
            np.random.shuffle(shuffled_arg)
            self.inputs = self.inputs[shuffled_arg]
            self.mask = self.mask[shuffled_arg]
            self.targets = self.targets[shuffled_arg]
        n_batch = int(self.length / batch_size)
        if self.length % batch_size != 0:
            n_batch += 1
        slices = np.split(np.arange(n_batch * batch_size), n_batch)
        slices[-1] = slices[-1][:(self.length - batch_size * (n_batch - 1))]
        return slices

    def get_slice(self, i):
        inputs, mask, targets = self.inputs[i], self.mask[i], self.targets[i]
        '''
        print("##### in get_slice, inputs")
        print(inputs)
        print(len(inputs),len(inputs[0]),len(inputs[1]))
        print("##### in get_slice, mask")
        print(mask)
        print("##### in get_slice, targets")
        print(targets)
        '''
        items, n_node, A, alias_inputs = [], [], [], []
        for u_input in inputs:
            n_node.append(len(np.unique(u_input))) 
        max_n_node = np.max(n_node) # 해당 slices 중 한 sequence 안에서 가질 수 있는 아이템의 종류의 최댓값
        # 0 패딩 후에 개수를 따지는 것이기 때문에 id 개수 +1 임.
        
        for u_input in inputs:
            node = np.unique(u_input)
            items.append(node.tolist() + (max_n_node - len(node)) * [0])
            u_A = np.zeros((max_n_node, max_n_node)) # 균일한 크기로 생성.
            for i in np.arange(len(u_input) - 1):
                if u_input[i + 1] == 0:
                    break
                u = np.where(node == u_input[i])[0][0]
                v = np.where(node == u_input[i + 1])[0][0]
                u_A[u][v] = 1
            u_sum_in = np.sum(u_A, 0)
            u_sum_in[np.where(u_sum_in == 0)] = 1
            u_A_in = np.divide(u_A, u_sum_in)
            u_sum_out = np.sum(u_A, 1)
            u_sum_out[np.where(u_sum_out == 0)] = 1
            u_A_out = np.divide(u_A.transpose(), u_sum_out)
            u_A = np.concatenate([u_A_in, u_A_out]).transpose()
            A.append(u_A)
            alias_inputs.append([np.where(node == i)[0][0] for i in u_input])
        return alias_inputs, A, items, mask, targets


In [ ]:
train_data, valid_data = split_validation(train_data, 0.1)
test_data = valid_data

In [ ]:
train_data1 = Data(train_data, shuffle=False)
test_data1 = Data(test_data, shuffle=False)

In [ ]:
class opt:
  def __init__(self,step,hidden_size,batch_size,nonhybrid=None):
    self.step=step
    self.hiddenSize = hidden_size
    #self.n_node = n_node
    self.batchSize = batch_size
    self.nonhybrid = nonhybrid
    self.lr = 0.001
    self.l2 = 1e-5
    self.lr_dc_step = 3
    self.lr_dc = 0.1
    self.epoch = 30
    self.patience = 10
    

In [ ]:
oopt=opt(step = 1,hidden_size=100,batch_size=100)

In [ ]:
n_node = 43098

In [ ]:
class GNN(tf.keras.layers.Layer):
    def __init__(self, hidden_size, step=1):
        super(GNN, self).__init__()
        self.step = step
        self.hidden_size = hidden_size
        self.input_size = hidden_size * 2
        self.gate_size = 3 * hidden_size
        # self.w_ih = Parameter(torch.Tensor(self.gate_size, self.input_size))
        self.stdv = 1/math.sqrt(self.hidden_size)
        self.w_ih = self.add_weight(shape=(self.input_size,self.gate_size),
                               initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))
        '''
        AttributeError: module 'keras.api._v2.keras.initializers' has no attribute 'Uniform'
        add_weight doesn't have the attribute named 'kernel_initializer'
        '''
        # self.w_hh = Parameter(torch.Tensor(self.gate_size, self.hidden_size))
        self.w_hh = self.add_weight(shape=(self.hidden_size,self.gate_size),
                               initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))
        # self.b_ih = Parameter(torch.Tensor(self.gate_size))
        self.b_ih = self.add_weight(shape=(self.gate_size,),
                               initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))
        
        # self.b_hh = Parameter(torch.Tensor(self.gate_size))
        self.b_hh = self.add_weight(shape=(self.gate_size,),
                               initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))
                
        # self.b_iah = Parameter(torch.Tensor(self.hidden_size))
        self.b_iah = self.add_weight(shape=(self.hidden_size,),
                               initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))
        # self.b_oah = Parameter(torch.Tensor(self.hidden_size))
        self.b_oah = self.add_weight(shape=(self.hidden_size,),
                               initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))

        # self.linear_edge_in = nn.Linear(self.hidden_size, self.hidden_size, bias=True)
        self.linear_edge_in = tf.keras.layers.Dense(self.hidden_size, 
                                                    use_bias=True,
                               kernel_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv),
                               bias_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))
        '''
        TypeError: ('Unknown keyword argument:', 'bias') - > use_bias=True in Dense
        '''
        # self.linear_edge_out = nn.Linear(self.hidden_size, self.hidden_size, bias=True)
        self.linear_edge_out = tf.keras.layers.Dense(self.hidden_size,
                                    use_bias=True ,
                               kernel_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv),
                               bias_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))
        # self.linear_edge_f = nn.Linear(self.hidden_size, self.hidden_size, bias=True)
        self.linear_edge_f = tf.keras.layers.Dense(self.hidden_size,
                                    use_bias=True ,
                               kernel_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv),
                               bias_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))

    def Cell(self, A, hidden):
        input_in = tf.matmul(A[:, :, :A.shape[1]], self.linear_edge_in(hidden)) + self.b_iah
        input_out = tf.matmul(A[:, :, A.shape[1]: 2 * A.shape[1]],self.linear_edge_out(hidden)) + self.b_oah
        inputs = tf.concat([input_in, input_out], 2) 
        gi = tf.matmul(inputs, self.w_ih )+ self.b_ih # Dense layer와 같은 역할함.
        # gate_size가 앞서서 3* hidden_size로 정의되어 있었고 아래 chunk에서 split하는 것 같다.
        gh = tf.matmul(hidden, self.w_hh) + self.b_hh
        i_r, i_i, i_n = tf.split(gi,3,axis = 2)
        h_r, h_i, h_n = tf.split(gh,3,axis = 2)
        resetgate = tf.nn.sigmoid(i_r + h_r)
        inputgate = tf.nn.sigmoid(i_i + h_i)
        newgate = tf.nn.tanh(i_n + resetgate * h_n)
        hy = newgate + inputgate * (hidden - newgate)
        return hy

    def call(self, A, hidden):
        for i in range(self.step):
            hidden = self.Cell(A, hidden)
        return hidden

In [ ]:

class TAGNN(tf.keras.models.Model):
    def __init__(self, opt, n_node):
        super(TAGNN, self).__init__()
        self.hidden_size = opt.hiddenSize
        self.n_node = n_node
        self.batch_size = opt.batchSize
        self.nonhybrid = opt.nonhybrid
        # self.embedding = nn.Embedding(self.n_node, self.hidden_size)
        self.embedding = tf.keras.layers.Embedding(input_dim = self.n_node,output_dim=self.hidden_size)
        self.stdv = 1/math.sqrt(self.hidden_size)
        print(self.stdv)
        self.gnn = GNN(self.hidden_size, step=opt.step)

        # self.layer_norm1 = nn.LayerNorm(self.hidden_size)
        self.layer_norm1 =tf.keras.layers.LayerNormalization(axis = -1)
        # self.attn = tf.keras.layers.MultiheadAttention( embed_dim=self.hidden_size, num_heads=2, dropout=0.1)
        self.attn = tf.keras.layers.MultiHeadAttention(num_heads=2,key_dim=self.hidden_size,dropout=0.1)

        # self.linear_one = nn.Linear(self.hidden_size, self.hidden_size, bias=True)
        self.linear_one = tf.keras.layers.Dense(self.hidden_size, use_bias=True,
                                                kernel_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv),
                                                bias_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))
        # self.linear_two = nn.Linear(self.hidden_size, self.hidden_size, bias=True)
        self.linear_two = tf.keras.layers.Dense(self.hidden_size, use_bias=True,
                                                kernel_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv),
                                                bias_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))
        # self.linear_three = nn.Linear(self.hidden_size, 1, bias=False)
        self.linear_three = tf.keras.layers.Dense(1 , use_bias=False,
                                                  kernel_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv),
                                                  )
        # self.linear_transform = nn.Linear(self.hidden_size * 2, self.hidden_size, bias=True)
        
        self.linear_transform = tf.keras.layers.Dense(self.hidden_size, use_bias=True,
                                                      kernel_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv),
                                                      bias_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv))

        # self.linear_t = nn.Linear(self.hidden_size, self.hidden_size, bias=False)  #target attention
        self.linear_t = tf.keras.layers.Dense( self.hidden_size, use_bias=False,
                                              kernel_initializer=tf.keras.initializers.RandomUniform(-self.stdv,self.stdv),
                                              )

    def call(self, inputs, A, mask,alias_inputs):
        hidden = self.embedding(inputs) 
        hidden = self.gnn(A, hidden)

        hidden = tf.transpose(hidden,[1,0,2])
        skip = self.layer_norm1(hidden)


        hidden, attn_w = self.attn(
            key = hidden, query = hidden, value = hidden,return_attention_scores=True)
            # hidden, hidden, hidden, attn_mask=get_mask(hidden.shape[0])) 원래 이런식으로 mask까지 해주어야 하는데
            # torch와 tensorflow mask 방식이 좀 달라서 공부 필요.
        hidden = hidden+skip
        hidden = tf.transpose(hidden,[1,0,2])
        
        get = lambda j: tf.gather(hidden[j],alias_inputs[j])
        hidden = tf.stack([get(j) for j in range(len(alias_inputs))])
        ht = tf.gather_nd(hidden ,indices = tf.stack([tf.range(mask.shape[0],dtype='int64') , tf.cast(tf.math.reduce_sum(mask,1),'int64') -1],axis=1))
        # tf.reshape을 이용해야함.
        q1 = tf.reshape(self.linear_one(ht) ,[ht.shape[0], 1, ht.shape[1]])  # batch_size x 1 x latent_size
        q2 = self.linear_two(hidden)  # batch_size x seq_length x latent_size
        alpha = self.linear_three(tf.nn.sigmoid(q1 + q2))  # (b,s,1)
        
        alpha = tf.nn.softmax(alpha, 1) # B,S,1
        
        
        a = tf.math.reduce_sum(alpha * hidden * tf.cast(tf.reshape(mask, [mask.shape[0], -1, 1]),dtype = 'float32'), 1)  # (b,d)
        if not self.nonhybrid: # 여기 적용되는지 판단할 것
        
            a = self.linear_transform(tf.concat([a, ht], 1)) 
        
        b = self.embedding.weights[0][1:] # 이 차원이 맞긴 함.

        hidden = hidden * tf.cast(tf.reshape(mask,[mask.shape[0], -1, 1]),'float32')  # batch_size x seq_length x latent_size
        

        qt = self.linear_t(hidden)  # batch_size x seq_length x latent_size

        beta = tf.nn.softmax(b @ tf.transpose(qt,[0,2,1]), -1)  # batch_size x n_nodes x seq_length
        target = beta @ hidden  # batch_size x n_nodes x latent_size
        a = tf.reshape(a,[ht.shape[0], 1, ht.shape[1]])  # b,1,d
        a = a + target  # b,n,d # 이거 concat인가 ? 

        scores = tf.math.reduce_sum(a * b, -1)  # b,n
        scores=tf.nn.softmax(scores,-1)
        return scores

In [ ]:
model=TAGNN(oopt,n_node)

loss_ftn = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate = oopt.lr)


#@tf.function
def train_ftn(item_input,A_input,mask_input,alias_inputs_,targets_input):
    with tf.GradientTape() as tape :
      scores = model(item_input, A_input, mask_input,alias_inputs_,training =True)
      
      ll = loss_ftn(targets_input-1,scores)

    
    
    gradients =tape.gradient(ll,model.trainable_weights)

    optimizer.apply_gradients(zip(gradients,model.trainable_weights))
    return ll

In [ ]:
%load_ext tensorboard

In [ ]:
import datetime

In [ ]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [ ]:
train_log_dir = './' + current_time + '/train'
tb_callback = tf.keras.callbacks.TensorBoard(train_log_dir)
tb_callback.set_model(model)

In [ ]:
for epoch in range(oopt.epoch):
    print('-------------------------------------------------------')
    print('epoch: ', epoch)

    slices = train_data1.generate_batch(model.batch_size)
    loss = 0
    for i, j in zip(slices, np.arange(len(slices))):
        alias_inputs, A, items, mask, targets = train_data1.get_slice(i)

        alias_inputs = tf.convert_to_tensor(alias_inputs)
        items = tf.convert_to_tensor(items)
        
        A = tf.convert_to_tensor(A)
        mask = tf.convert_to_tensor(mask)
        targets = tf.convert_to_tensor(targets)
        
        l = train_ftn(items,A,mask,alias_inputs,targets)
        
        loss += l 
        
        if j % int(100) == 0:
            print('[%d/%d] Loss: %.4f' % (j, len(slices), l))
    print('\tLoss:\t%.3f' % loss)
    hit, mrr = [], []
    slices = test_data1.generate_batch(model.batch_size)
    for i in slices:
        alias_inputs, A, items, mask, targets = test_data1.get_slice(i)
        
        alias_inputs = tf.convert_to_tensor(alias_inputs)
        items = tf.convert_to_tensor(items)
        
        A = tf.convert_to_tensor(A)
        mask = tf.convert_to_tensor(mask)
        targets = tf.convert_to_tensor(targets)

        scores = model(items, A, mask,alias_inputs,training =False)
        scores = tf.math.top_k(scores,k=20).indices
        scores = scores.numpy()
        
            # sub_scores를 tensor에서 numpy로 바꾸고 아래는 그대로 실행하면 됨.
        for score, target, mask in zip(scores, targets, test_data1.mask):
            hit.append(np.isin(target - 1, score)) # 현재 target은 s_{n+1}인 item이므로, 
                # score 즉, 위에서 추출한 index가 일치하면 hit이라는 list에 append 해준다.

                # score vector에서 target-1과 일치하는 경우를 찾고, 없는 경우 0을 mrr이라는 list에 기록하고,
            if len(np.where(score == target - 1)[0]) == 0:
                mrr.append(0)
            else:
                # 있는 경우에는 (1/그 위치의 index +1)을 기록한다.
                mrr.append(1 / (np.where(score == target - 1)[0][0] + 1))
    hit = np.mean(hit) * 100
    mrr = np.mean(mrr) * 100

    print(f"Recall@20 : {hit} , MRR@20 {mrr}")

```python
-------------------------------------------------------
epoch:  0
[0/6476] Loss: 10.6295
[100/6476] Loss: 10.3150
[200/6476] Loss: 10.0438
[300/6476] Loss: 9.8708
[400/6476] Loss: 9.4667
[500/6476] Loss: 8.5696
[600/6476] Loss: 8.5144
[700/6476] Loss: 8.2501
[800/6476] Loss: 8.2630
[900/6476] Loss: 7.8947
[1000/6476] Loss: 7.5036
[1100/6476] Loss: 6.9145
[1200/6476] Loss: 7.1480
[1300/6476] Loss: 6.7984
[1400/6476] Loss: 6.0888
[1500/6476] Loss: 7.2060
[1600/6476] Loss: 6.9092
[1700/6476] Loss: 6.4674
[1800/6476] Loss: 6.6813
[1900/6476] Loss: 7.2124
[2000/6476] Loss: 6.4806
[2100/6476] Loss: 6.5711
[2200/6476] Loss: 6.3636
[2300/6476] Loss: 5.9528
[2400/6476] Loss: 6.9861
[2500/6476] Loss: 6.4766
[2600/6476] Loss: 6.1471
[2700/6476] Loss: 5.5757
[2800/6476] Loss: 6.3495
[2900/6476] Loss: 5.6274
[3000/6476] Loss: 6.2108
[3100/6476] Loss: 6.1135
[3200/6476] Loss: 5.8157
[3300/6476] Loss: 5.6654
[3400/6476] Loss: 5.6421
[3500/6476] Loss: 5.6513
[3600/6476] Loss: 5.2236
[3700/6476] Loss: 5.5584
[3800/6476] Loss: 5.5214
[3900/6476] Loss: 5.8830
[4000/6476] Loss: 5.4331
[4100/6476] Loss: 5.4109
[4200/6476] Loss: 4.7986
[4300/6476] Loss: 5.3059
[4400/6476] Loss: 5.4154
[4500/6476] Loss: 5.5884
[4600/6476] Loss: 5.4203
[4700/6476] Loss: 5.9706
[4800/6476] Loss: 6.0336
[4900/6476] Loss: 5.4578
[5000/6476] Loss: 5.6494
[5100/6476] Loss: 5.4651
[5200/6476] Loss: 5.7962
[5300/6476] Loss: 5.1694
[5400/6476] Loss: 4.9791
[5500/6476] Loss: 5.5158
[5600/6476] Loss: 5.5360
[5700/6476] Loss: 5.4237
[5800/6476] Loss: 5.4118
[5900/6476] Loss: 5.4357
[6000/6476] Loss: 5.2031
[6100/6476] Loss: 5.7668
[6200/6476] Loss: 5.4463
[6300/6476] Loss: 5.5769
[6400/6476] Loss: 5.4836
	Loss:	41501.449
Recall@20 : 51.55322668075111 , MRR@20 17.807422059276096
-------------------------------------------------------
epoch:  1
[0/6476] Loss: 5.1545
[100/6476] Loss: 5.4708
[200/6476] Loss: 5.4562
[300/6476] Loss: 4.9342
[400/6476] Loss: 4.8897
[500/6476] Loss: 4.9473
[600/6476] Loss: 4.9085
[700/6476] Loss: 5.1303
[800/6476] Loss: 5.5906
[900/6476] Loss: 5.2830
[1000/6476] Loss: 4.7529
[1100/6476] Loss: 4.6014
[1200/6476] Loss: 5.0933
[1300/6476] Loss: 4.8952
[1400/6476] Loss: 4.5311
[1500/6476] Loss: 4.9976
[1600/6476] Loss: 5.3019
[1700/6476] Loss: 4.6897
[1800/6476] Loss: 5.1289
[1900/6476] Loss: 5.5528
[2000/6476] Loss: 4.5748
[2100/6476] Loss: 5.1159
[2200/6476] Loss: 5.0950
[2300/6476] Loss: 4.8364
[2400/6476] Loss: 5.8616
[2500/6476] Loss: 5.0987
[2600/6476] Loss: 5.1137
[2700/6476] Loss: 4.6788
[2800/6476] Loss: 5.0722
[2900/6476] Loss: 4.6408
[3000/6476] Loss: 5.3349
[3100/6476] Loss: 4.9262
[3200/6476] Loss: 4.8729
[3300/6476] Loss: 4.6849
[3400/6476] Loss: 4.9103
[3500/6476] Loss: 4.9671
[3600/6476] Loss: 4.4253
[3700/6476] Loss: 4.8307
[3800/6476] Loss: 4.9069
[3900/6476] Loss: 5.1425
[4000/6476] Loss: 4.7411
[4100/6476] Loss: 4.7446
[4200/6476] Loss: 4.2579
[4300/6476] Loss: 4.6920
[4400/6476] Loss: 4.8863
[4500/6476] Loss: 4.9109
[4600/6476] Loss: 4.8870
[4700/6476] Loss: 5.3142
[4800/6476] Loss: 5.5017
[4900/6476] Loss: 5.0037
[5000/6476] Loss: 5.1335
[5100/6476] Loss: 4.9857
[5200/6476] Loss: 5.2525
[5300/6476] Loss: 4.7210
[5400/6476] Loss: 4.5055
[5500/6476] Loss: 4.8940
[5600/6476] Loss: 4.8881
[5700/6476] Loss: 4.8758
[5800/6476] Loss: 5.0788
[5900/6476] Loss: 4.8038
[6000/6476] Loss: 4.7509
[6100/6476] Loss: 5.2443
[6200/6476] Loss: 5.0295
[6300/6476] Loss: 5.2089
[6400/6476] Loss: 4.9150
	Loss:	32288.967
Recall@20 : 52.59705060669659 , MRR@20 18.56622950806902
-------------------------------------------------------
epoch:  2
[0/6476] Loss: 4.8443
[100/6476] Loss: 4.8576
[200/6476] Loss: 4.9325
[300/6476] Loss: 4.4477
[400/6476] Loss: 4.4532
[500/6476] Loss: 4.5142
[600/6476] Loss: 4.4772
[700/6476] Loss: 4.7887
[800/6476] Loss: 5.0661
[900/6476] Loss: 4.9145
[1000/6476] Loss: 4.4217
[1100/6476] Loss: 4.1679
[1200/6476] Loss: 4.8126
[1300/6476] Loss: 4.5822
[1400/6476] Loss: 4.1527
[1500/6476] Loss: 4.5450
[1600/6476] Loss: 4.8734
[1700/6476] Loss: 4.2996
[1800/6476] Loss: 4.8043
[1900/6476] Loss: 5.1285
[2000/6476] Loss: 4.2490
[2100/6476] Loss: 4.6021
[2200/6476] Loss: 4.8195
[2300/6476] Loss: 4.3530
[2400/6476] Loss: 5.4434
[2500/6476] Loss: 4.6763
[2600/6476] Loss: 4.7689
[2700/6476] Loss: 4.4170
[2800/6476] Loss: 4.5984
[2900/6476] Loss: 4.3268
[3000/6476] Loss: 4.8692
[3100/6476] Loss: 4.5012
[3200/6476] Loss: 4.4860
[3300/6476] Loss: 4.3039
[3400/6476] Loss: 4.6910
[3500/6476] Loss: 4.6607
[3600/6476] Loss: 4.0912
[3700/6476] Loss: 4.4766
[3800/6476] Loss: 4.6669
[3900/6476] Loss: 4.7070
[4000/6476] Loss: 4.3723
[4100/6476] Loss: 4.3317
[4200/6476] Loss: 3.9546
[4300/6476] Loss: 4.3774
[4400/6476] Loss: 4.5565
[4500/6476] Loss: 4.5274
[4600/6476] Loss: 4.6427
[4700/6476] Loss: 4.7245
[4800/6476] Loss: 5.0460
[4900/6476] Loss: 4.6685
[5000/6476] Loss: 4.8185
[5100/6476] Loss: 4.7379
[5200/6476] Loss: 4.6792
[5300/6476] Loss: 4.4288
[5400/6476] Loss: 4.1703
[5500/6476] Loss: 4.4255
[5600/6476] Loss: 4.4952
[5700/6476] Loss: 4.4789
[5800/6476] Loss: 4.8839
[5900/6476] Loss: 4.4123
[6000/6476] Loss: 4.4523
[6100/6476] Loss: 4.9224
[6200/6476] Loss: 4.6822
[6300/6476] Loss: 4.8448
[6400/6476] Loss: 4.6071
	Loss:	29816.891
Recall@20 : 50.751247446036665 , MRR@20 17.97607031920885
-------------------------------------------------------
epoch:  3
[0/6476] Loss: 4.6688
[100/6476] Loss: 4.2132
[200/6476] Loss: 4.4587
[300/6476] Loss: 4.1034
[400/6476] Loss: 4.0549
[500/6476] Loss: 4.2595
[600/6476] Loss: 4.1308
[700/6476] Loss: 4.4733
[800/6476] Loss: 4.6030
[900/6476] Loss: 4.5947
[1000/6476] Loss: 4.1559
[1100/6476] Loss: 3.9065
[1200/6476] Loss: 4.6512
[1300/6476] Loss: 4.3107
[1400/6476] Loss: 3.8081
[1500/6476] Loss: 4.2585
[1600/6476] Loss: 4.5161
[1700/6476] Loss: 3.9902
[1800/6476] Loss: 4.5709
[1900/6476] Loss: 4.7187
[2000/6476] Loss: 3.8890
[2100/6476] Loss: 4.2237
[2200/6476] Loss: 4.5519
[2300/6476] Loss: 3.9362
[2400/6476] Loss: 5.1873
[2500/6476] Loss: 4.4063
[2600/6476] Loss: 4.5359
[2700/6476] Loss: 4.1022
[2800/6476] Loss: 4.3655
[2900/6476] Loss: 4.0535
[3000/6476] Loss: 4.4701
[3100/6476] Loss: 4.1601
[3200/6476] Loss: 4.1002
[3300/6476] Loss: 4.0185
[3400/6476] Loss: 4.3648
[3500/6476] Loss: 4.3732
[3600/6476] Loss: 3.8115
[3700/6476] Loss: 4.1050
[3800/6476] Loss: 4.3066
[3900/6476] Loss: 4.3702
[4000/6476] Loss: 4.0275
[4100/6476] Loss: 4.0584
[4200/6476] Loss: 3.7700
[4300/6476] Loss: 4.1468
[4400/6476] Loss: 4.3464
[4500/6476] Loss: 4.2379
[4600/6476] Loss: 4.4144
[4700/6476] Loss: 4.3382
[4800/6476] Loss: 4.6189
[4900/6476] Loss: 4.3790
[5000/6476] Loss: 4.5152
[5100/6476] Loss: 4.5095
[5200/6476] Loss: 4.3219
[5300/6476] Loss: 4.2176
[5400/6476] Loss: 3.8635
[5500/6476] Loss: 4.1795
[5600/6476] Loss: 4.1896
[5700/6476] Loss: 4.2474
[5800/6476] Loss: 4.6574
[5900/6476] Loss: 4.0820
[6000/6476] Loss: 4.2612
[6100/6476] Loss: 4.6720
[6200/6476] Loss: 4.5061
[6300/6476] Loss: 4.5560
[6400/6476] Loss: 4.3027
	Loss:	27787.273
Recall@20 : 48.76506317150125 , MRR@20 17.13053096707961
-------------------------------------------------------
epoch:  4
[0/6476] Loss: 4.5365
[100/6476] Loss: 3.7999
[200/6476] Loss: 4.0814
[300/6476] Loss: 3.7685
[400/6476] Loss: 3.6887
[500/6476] Loss: 4.0764
[600/6476] Loss: 3.9550
[700/6476] Loss: 4.2862
[800/6476] Loss: 4.3043
[900/6476] Loss: 4.2745
[1000/6476] Loss: 3.9898
[1100/6476] Loss: 3.7635
[1200/6476] Loss: 4.3244
[1300/6476] Loss: 4.1041
[1400/6476] Loss: 3.5847
[1500/6476] Loss: 4.0195
[1600/6476] Loss: 4.1587
[1700/6476] Loss: 3.7167
[1800/6476] Loss: 4.2938
[1900/6476] Loss: 4.4170
[2000/6476] Loss: 3.6152
[2100/6476] Loss: 4.0570
[2200/6476] Loss: 4.2910
[2300/6476] Loss: 3.6186
[2400/6476] Loss: 5.0287
[2500/6476] Loss: 4.0977
[2600/6476] Loss: 4.3822
[2700/6476] Loss: 3.8527
[2800/6476] Loss: 4.3587
[2900/6476] Loss: 3.9589
[3000/6476] Loss: 4.2329
[3100/6476] Loss: 3.9583
[3200/6476] Loss: 3.8525
[3300/6476] Loss: 3.6958
[3400/6476] Loss: 4.0743
[3500/6476] Loss: 4.1485
[3600/6476] Loss: 3.6974
[3700/6476] Loss: 3.8464
[3800/6476] Loss: 4.1807
[3900/6476] Loss: 4.1727
[4000/6476] Loss: 3.8333
[4100/6476] Loss: 3.8129
[4200/6476] Loss: 3.6569
[4300/6476] Loss: 3.9606
[4400/6476] Loss: 4.1515
[4500/6476] Loss: 4.0689
[4600/6476] Loss: 4.2008
[4700/6476] Loss: 4.1116
[4800/6476] Loss: 4.3792
[4900/6476] Loss: 4.1358
[5000/6476] Loss: 4.1964
[5100/6476] Loss: 4.2951
[5200/6476] Loss: 4.2247
[5300/6476] Loss: 4.0245
[5400/6476] Loss: 3.6598
[5500/6476] Loss: 4.0905
[5600/6476] Loss: 4.0009
[5700/6476] Loss: 3.9206
[5800/6476] Loss: 4.4747
[5900/6476] Loss: 3.7334
[6000/6476] Loss: 4.0851
[6100/6476] Loss: 4.3217
[6200/6476] Loss: 4.2636
[6300/6476] Loss: 4.3252
[6400/6476] Loss: 4.1542
	Loss:	26234.795
Recall@20 : 46.69826399988881 , MRR@20 16.32665875009864
-------------------------------------------------------
```